In [1]:
MACCS_best_cat_params={
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS',
 'depth': 10,
 'iterations': 900,
 'l2_leaf_reg': 0.2557273602325932,
 'leaf_estimation_method': 'Gradient',
 'learning_rate': 0.0477685119153238,
 'random_strength': 0.0061100684626963,
 'rsm': 0.6847953796328295,
 'score_function': 'L2',
 'early_stopping_rounds':20,
 'eval_metric':'R2'}
MACCS_best_xgb_params={'alpha': 0.0014906977558625,
 'booster': 'dart',
 'colsample_bytree': 0.8546173904005345,
 'eta': 0.0456833498124193,
 'gamma': 3.3542988420770624e-05,
 'grow_policy': 'depthwise',
 'lambda': 0.2943003047107695,
 'max_depth': 14,
 'min_child_weight': 10,
 'n_estimators': 220,
 'normalize_type': 'forest',
 'rate_drop': 0.024216902211587,
 'sample_type': 'weighted',
 'skip_drop': 0.001261004949685,
 'subsample': 0.7218018660663547,
 'random_state':42,
 'verbosity': 0,
 'objective': 'reg:squarederror'}
MD_best_xgb_params={'alpha': 5.965346832119681e-06,
 'booster': 'gbtree',
 'verbosity': 0,
 'tree_method': 'exact',
 'objective': 'reg:squarederror',
 'colsample_bytree': 0.7295911276220814,
 'gamma': 7.372146242868913e-08,
 'grow_policy': 'lossguide',
 'lambda': 0.0002210864159401,
 'max_depth': 9,
 'eta':0.05,
 'min_child_weight': 4,
 'n_estimators': 220,
 'normalize_type': None,
 'rate_drop':None,
 'sample_type':None,
 'skip_drop': None,
 'subsample': 0.926368451515968,
 'random_state':42}
MD_best_cat_params={'iterations': 1000, 
'learning_rate': 0.05195087349854926,
'depth': 10, 
'l2_leaf_reg': 0.8124329970418442,
'boosting_type': 'Ordered',
'bootstrap_type': 'MVS',
'random_strength': 0.00015070897262938095, 
'score_function': 'L2', 
'leaf_estimation_method':"Gradient",
'rsm': 0.3037411535337428,
'early_stopping_rounds':20,
'eval_metric':'R2'}

In [2]:
import pandas as pd
from mordred import Calculator, descriptors
from rdkit import Chem,DataStructs
from rdkit.Chem import AllChem,PandasTools
import rdkit
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, balanced_accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV, train_test_split
import xgboost as xgb
import catboost as cat
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
from rdkit.Chem import rdFingerprintGenerator
import graphviz

In [3]:
df=pd.read_csv(r"C:\Users\19189\Desktop\original_data.csv")
df.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199


In [4]:
def MACCSfp(mol):
    fp = rdkit.Chem.rdMolDescriptors.GetMACCSKeysFingerprint(mol)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

In [5]:
PandasTools.AddMoleculeColumnToFrame(df,'Canonical_QSARr','Molecule')
df.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50,Molecule
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758,<rdkit.Chem.rdchem.Mol object at 0x000001B8E68...
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199,<rdkit.Chem.rdchem.Mol object at 0x000001B8E68...


In [6]:
df["MACCS_FPs"] = df.Molecule.apply(MACCSfp)
df.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50,Molecule,MACCS_FPs
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758,<rdkit.Chem.rdchem.Mol object at 0x000001B8E68...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199,<rdkit.Chem.rdchem.Mol object at 0x000001B8E68...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
MACCS_fp_df = df["MACCS_FPs"].apply(pd.Series)
MACCS_fp_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,1,1,0


In [8]:
MD=pd.read_csv(r"C:\Users\19189\Desktop\mordred_descriptors.csv")
MD.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758,0.0,0.0,0,0,34.992970,2.613976,...,10.344674,84.588084,416.069448,9.044988,2244,39,146.0,172.0,10.201389,6.166667
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199,0.0,0.0,0,0,11.643052,2.307250,...,9.078065,39.748909,156.034193,8.212326,110,13,48.0,54.0,4.944444,2.222222


In [9]:
MD_x=MD.iloc[:,4:1831]
MD_x =MD_x.astype(float)
MD_x.head(2)

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0.0,0.0,0.0,0.0,34.992970,2.613976,4.943050,34.992970,1.249749,4.277221,...,10.344674,84.588084,416.069448,9.044988,2244.0,39.0,146.0,172.0,10.201389,6.166667
1,0.0,0.0,0.0,0.0,11.643052,2.307250,4.614501,11.643052,1.164305,3.206363,...,9.078065,39.748909,156.034193,8.212326,110.0,13.0,48.0,54.0,4.944444,2.222222


In [10]:
MACCS_x = MACCS_fp_df.iloc[:,0:167]
y = df['LogLD50']
type(y)

pandas.core.series.Series

In [11]:
data=pd.concat([MACCS_x,MD_x],axis=1)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0,0,0,0,0,0,0,0,0,...,10.344674,84.588084,416.069448,9.044988,2244.0,39.0,146.0,172.0,10.201389,6.166667
1,0,0,0,0,0,0,0,0,0,0,...,9.078065,39.748909,156.034193,8.212326,110.0,13.0,48.0,54.0,4.944444,2.222222


In [12]:
data=pd.concat([data,y],axis=1)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,LogLD50
0,0,0,0,0,0,0,0,0,0,0,...,84.588084,416.069448,9.044988,2244.0,39.0,146.0,172.0,10.201389,6.166667,2.662758
1,0,0,0,0,0,0,0,0,0,0,...,39.748909,156.034193,8.212326,110.0,13.0,48.0,54.0,4.944444,2.222222,3.583199


In [13]:
y = data['LogLD50']

In [14]:
x = data.iloc[:,0:1993]

In [15]:
x_train, xtest, y_train, ytest = train_test_split(x, y, test_size=0.4,random_state=42)

In [16]:
x_test, x_val, y_test, y_val= train_test_split(xtest, ytest, test_size=0.2,random_state=42)

In [53]:
y_val

3107     3.653213
3372     3.451018
3487     3.544068
9726     2.638489
450      2.515874
           ...   
6925     2.845098
9866     3.602060
10915    3.000000
8176     3.606381
9070     2.397940
Name: LogLD50, Length: 959, dtype: float64

In [72]:
d1=x_test.iloc[:,0:1993]

In [73]:
data1=pd.concat([x_train,y_train],axis=1)
data2=pd.concat([d1,y_test],axis=1)

In [78]:
train_dataset=pd.concat([data1,data2])
train_dataset

,0,1,2,3,4,5,6,7,8,9,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,LogLD50
7735,0,0,0,0,0,0,0,0,0,0,...,81.187037,425.235479,6.749770,3147.0,50.0,170.0,201.0,7.500000,7.138889,2.480007
5545,0,0,0,0,0,0,0,0,0,0,...,40.723493,149.120449,5.735402,160.0,10.0,52.0,54.0,4.673611,2.375000,2.419956
11654,0,0,0,0,0,0,0,0,0,0,...,60.632208,365.183838,6.890261,1552.0,42.0,126.0,147.0,9.888889,6.111111,3.544068
1168,0,0,0,0,0,0,0,0,0,0,...,45.785530,206.094294,7.106700,434.0,16.0,68.0,72.0,6.194444,3.416667,3.482588
4144,0,0,0,0,0,0,0,0,0,0,...,50.743793,261.096085,7.912003,573.0,30.0,84.0,99.0,8.166667,4.416667,3.804480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,0,0,0,0,0,0,0,0,...,67.599297,287.199762,6.243473,1039.0,28.0,100.0,114.0,6.805556,5.055556,3.146128
2919,0,0,0,0,0,0,0,0,0,0,...,41.829266,168.151415,5.254732,211.0,13.0,54.0,58.0,4.833333,2.833333,3.875061
1921,0,0,0,0,0,0,0,0,0,0,...,60.196658,174.127726,6.004404,208.0,23.0,72.0,89.0,5.506944,2.750000,2.849419
1047,0,0,0,0,0,0,0,0,0,0,...,49.437854,313.971549,10.826605,544.0,25.0,82.0,93.0,8.006944,3.930556,1.812913


In [79]:
train_dataset.to_csv(r'C:\Users\19189\Desktop\train_set.csv',index=False)

In [75]:
data2

,0,1,2,3,4,5,6,7,8,9,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,LogLD50
10196,0,0,0,0,0,0,0,0,0,0,...,78.666404,388.370516,5.110138,2022.0,54.0,158.0,195.0,10.013889,5.916667,3.176091
9364,0,0,0,0,0,0,0,0,0,0,...,42.543530,234.074822,7.314838,433.0,11.0,54.0,52.0,6.222222,3.416667,2.702431
10441,0,0,0,0,0,0,0,0,0,0,...,48.368390,217.073893,8.039774,426.0,24.0,80.0,93.0,5.555556,3.666667,2.472756
582,0,0,0,0,0,0,0,0,0,0,...,65.588748,241.085127,8.313280,544.0,30.0,98.0,119.0,5.777778,3.916667,3.000000
8412,0,0,0,0,0,0,0,0,0,0,...,68.238023,331.010187,9.735594,819.0,32.0,102.0,121.0,7.868056,4.597222,1.698970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,0,0,0,0,0,0,0,0,...,67.599297,287.199762,6.243473,1039.0,28.0,100.0,114.0,6.805556,5.055556,3.146128
2919,0,0,0,0,0,0,0,0,0,0,...,41.829266,168.151415,5.254732,211.0,13.0,54.0,58.0,4.833333,2.833333,3.875061
1921,0,0,0,0,0,0,0,0,0,0,...,60.196658,174.127726,6.004404,208.0,23.0,72.0,89.0,5.506944,2.750000,2.849419
1047,0,0,0,0,0,0,0,0,0,0,...,49.437854,313.971549,10.826605,544.0,25.0,82.0,93.0,8.006944,3.930556,1.812913


In [52]:
val=pd.concat([x_val,y_val],axis=1)
val.head(2)

,0,1,2,3,4,5,6,7,8,9,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,LogLD50
3107,0,0,0,0,0,0,0,0,0,0,...,63.430691,261.045964,9.001585,698.0,21.0,88.0,98.0,6.055556,4.027778,3.653213
3372,0,0,0,0,0,0,0,0,0,0,...,40.037455,142.135765,5.076277,109.0,11.0,50.0,54.0,5.034722,2.083333,3.451018


In [54]:
val.to_csv(r'C:\Users\19189\Desktop\val_set.csv',index=False)

In [17]:
models = [xgb.XGBRegressor(**MACCS_best_xgb_params),
          cat.CatBoostRegressor(**MACCS_best_cat_params),
          xgb.XGBRegressor(**MD_best_xgb_params),
          cat.CatBoostRegressor(**MD_best_cat_params)]
model_names = ['X_MACCS', 'C_MACCS','X_MD', 'C_MD']
for idx, model in enumerate(models):
    if idx<=1:
        xtest=x_test.iloc[:,0:167]
        xtrain=x_train.iloc[:,0:167]
    else:
        xtest=x_test.iloc[:,167:1993]
        xtrain=x_train.iloc[:,167:1993]
    model.fit(xtrain, y_train)
    if idx%2==0:
        file_name=r'C:\Users\19189\Desktop\{0}.json'.format(model_names[idx])
    else:
        file_name=r'C:\Users\19189\Desktop\{0}.model'.format(model_names[idx])
    model.save_model(file_name)  
    y_pred = model.predict(xtest)
    x_test[str(model_names[idx])]=y_pred
x_test['LogLD50']=y_test
x_test.head(2)  
    

0:	learn: 0.0352033	total: 292ms	remaining: 4m 22s
1:	learn: 0.0657393	total: 449ms	remaining: 3m 21s
2:	learn: 0.0961848	total: 601ms	remaining: 2m 59s
3:	learn: 0.1224760	total: 750ms	remaining: 2m 48s
4:	learn: 0.1464854	total: 908ms	remaining: 2m 42s
5:	learn: 0.1686772	total: 1.08s	remaining: 2m 41s
6:	learn: 0.1890763	total: 1.24s	remaining: 2m 37s
7:	learn: 0.2084652	total: 1.39s	remaining: 2m 35s
8:	learn: 0.2274397	total: 1.56s	remaining: 2m 34s
9:	learn: 0.2445457	total: 1.71s	remaining: 2m 32s
10:	learn: 0.2615273	total: 1.87s	remaining: 2m 31s
11:	learn: 0.2772594	total: 2.02s	remaining: 2m 29s
12:	learn: 0.2910569	total: 2.19s	remaining: 2m 29s
13:	learn: 0.3050003	total: 2.34s	remaining: 2m 28s
14:	learn: 0.3160546	total: 2.5s	remaining: 2m 27s
15:	learn: 0.3283579	total: 2.65s	remaining: 2m 26s
16:	learn: 0.3385533	total: 2.81s	remaining: 2m 25s
17:	learn: 0.3480551	total: 2.97s	remaining: 2m 25s
18:	learn: 0.3605057	total: 3.14s	remaining: 2m 25s
19:	learn: 0.3712587	to

,0,1,2,3,4,5,6,7,8,9,...,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,X_MACCS,C_MACCS,X_MD,C_MD,LogLD50
10196,0,0,0,0,0,0,0,0,0,0,...,54.0,158.0,195.0,10.013889,5.916667,3.602569,3.495990,3.525380,3.444286,3.176091
9364,0,0,0,0,0,0,0,0,0,0,...,11.0,54.0,52.0,6.222222,3.416667,3.142085,3.153181,3.309242,3.415576,2.702431


In [25]:
#ccc = cat.CatBoostRegressor()
#ccc.load_model(r"C:\Users\19189\Desktop\C_MACCS.model")

In [21]:
#ccc.plot_tree(1)

In [27]:
pd.set_option('display.max_rows', None)

In [22]:
#ccc.get_feature_importance(prettified=True)

In [59]:
#model_xgb_2 = xgb.Booster()
#model_xgb_2.load_model(r"C:\Users\19189\Desktop\X_MACCS.json")

In [58]:
#xgb.plot_importance(model_xgb_2, max_num_features=10)

In [24]:
#xgb.to_graphviz(model_xgb_2)

In [18]:
df=x_test[['C_MACCS','X_MACCS','C_MD','X_MD','LogLD50']]
df.head(2)

,C_MACCS,X_MACCS,C_MD,X_MD,LogLD50
10196,3.495990,3.602569,3.444286,3.525380,3.176091
9364,3.153181,3.142085,3.415576,3.309242,2.702431


In [20]:
pd.options.mode.copy_on_write = True

In [21]:
df["values"] =df[['C_MACCS', 'X_MACCS', 'C_MD', 'X_MD']].values.tolist()

In [22]:
df

,C_MACCS,X_MACCS,C_MD,X_MD,LogLD50,values
10196,3.495990,3.602569,3.444286,3.525380,3.176091,"[3.4959898114765546, 3.6025686264038086, 3.444..."
9364,3.153181,3.142085,3.415576,3.309242,2.702431,"[3.153180991659444, 3.142085075378418, 3.41557..."
10441,3.062262,2.837857,2.877951,2.957898,2.472756,"[3.0622620251910653, 2.837857484817505, 2.8779..."
582,3.202319,3.033181,2.832310,2.694986,3.000000,"[3.202319407918886, 3.0331807136535645, 2.8323..."
8412,1.757492,1.666688,1.699257,1.568344,1.698970,"[1.7574918947272284, 1.6666884422302246, 1.699..."
...,...,...,...,...,...,...
199,3.318453,3.312856,2.968701,2.939271,3.146128,"[3.318452861330711, 3.3128561973571777, 2.9687..."
2919,3.571349,3.629738,3.531822,3.684559,3.875061,"[3.571349241281328, 3.6297380924224854, 3.5318..."
1921,2.519334,2.579201,2.670275,2.780704,2.849419,"[2.5193341825906144, 2.5792009830474854, 2.670..."
1047,2.106975,2.123811,1.957155,1.684854,1.812913,"[2.1069753110348257, 2.1238107681274414, 1.957..."


In [33]:
df1=df.reset_index(drop=True)

In [34]:
df1.head(2)

,C_MACCS,X_MACCS,C_MD,X_MD,LogLD50,values
0,3.495990,3.602569,3.444286,3.525380,3.176091,"[3.4959898114765546, 3.6025686264038086, 3.444..."
1,3.153181,3.142085,3.415576,3.309242,2.702431,"[3.153180991659444, 3.142085075378418, 3.41557..."


In [35]:
from rdkit import Chem,DataStructs
from rdkit.Chem import AllChem,PandasTools
import rdkit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,random_split
from rdkit import Chem
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [36]:
class VDataset(torch.utils.data.Dataset):
    def __init__(self, values_list, labels):
        self.values_list = values_list
        self.labels = labels

    def __len__(self):
        return len(self.values_list)

    def __getitem__(self, idx):
        value = self.values_list[idx]
        label = self.labels[idx]
        return torch.Tensor(value), torch.Tensor([label])

In [37]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.activation1 = nn.ReLU()
        self.activation2 = nn.Sigmoid()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.activation1(out)
        out = self.fc2(out)
        out = self.activation1(out)
        out = self.fc3(out)
        return out

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [47]:
input_size = 4  # Size of Morgan fingerprint
hidden_size = 15
output_size = 1
# Define loss function and optimizer
criterion = nn.MSELoss()

value = df1["values"]
labels = df1["LogLD50"]

# Create a custom dataset
dataset = VDataset(value, labels)
# Create a data loader

generator2 = torch.Generator().manual_seed(42)
train_dataset, test_dataset=random_split(dataset, [0.7,0.3], generator=generator2)


num_epochs = 285
lr=0.001
batch_size=1
data_loader1= DataLoader(train_dataset, batch_size=1, shuffle=True)
data_loader2= DataLoader(test_dataset, batch_size=1, shuffle=True)
model = Net(input_size, hidden_size, output_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(num_epochs):
    train_loss=0
    val_loss=0
    for batch_inputs, batch_labels in data_loader1:
        # Move the inputs and labels to the GPU
        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    for batch_inputs, batch_labels in data_loader2:
        # Move the inputs and labels to the GPU
        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        val_loss += loss.item()
    print("Epoch [{}/{}], Loss: {:.4f},{:.4f}".format(epoch+1, num_epochs, train_loss,val_loss))
torch.save(model.state_dict(), r"C:\Users\19189\Desktop\FCNN_model.pt")
print("Model saved!")

Epoch [1/285], Loss: 1441.3519,338.4225
Epoch [2/285], Loss: 766.5941,341.6426
Epoch [3/285], Loss: 763.5551,344.8460
Epoch [4/285], Loss: 751.5055,359.6961
Epoch [5/285], Loss: 757.8481,339.9702
Epoch [6/285], Loss: 754.3425,338.5602
Epoch [7/285], Loss: 753.2932,343.6053
Epoch [8/285], Loss: 747.1187,337.7507
Epoch [9/285], Loss: 750.5714,353.5865
Epoch [10/285], Loss: 744.7345,367.1891
Epoch [11/285], Loss: 751.2189,341.2988
Epoch [12/285], Loss: 752.5243,343.2578
Epoch [13/285], Loss: 748.9330,364.3907
Epoch [14/285], Loss: 748.6588,353.5945
Epoch [15/285], Loss: 740.6926,341.6995
Epoch [16/285], Loss: 751.2148,359.8973
Epoch [17/285], Loss: 746.0364,365.6479
Epoch [18/285], Loss: 746.8820,342.4245
Epoch [19/285], Loss: 749.8083,340.4642
Epoch [20/285], Loss: 748.0302,337.4204
Epoch [21/285], Loss: 746.4629,337.6610
Epoch [22/285], Loss: 742.4255,333.7514
Epoch [23/285], Loss: 746.0567,339.8105
Epoch [24/285], Loss: 747.6047,365.4270
Epoch [25/285], Loss: 739.8905,337.1288
Epoch [2

In [48]:
input_size = 4 
hidden_size = 15
output_size = 1
loaded_model = Net(input_size, hidden_size, output_size).to(device)
loaded_model.load_state_dict(torch.load(r"C:\Users\19189\Desktop\FCNN_model.pt"))
loaded_model.eval()
print("Model loaded!")

Model loaded!


In [49]:
with torch.no_grad():
    loaded_model.eval()
    output = loaded_model(input_tensor)
    prediction = output.item()

NameError: name 'input_tensor' is not defined